In [1]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions
using MacroTools

In [2]:
@model function smoothing(n, x0, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)
    c = constvar(1.0)

    x_prev = x_prior

    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

smoothing (generic function with 1 method)

In [3]:
P = 1.0

n = 10_000
k = 500
data = collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [4]:
function inference(; data, k, x0, P)
    n = length(data)
    
    _, (x, y) = smoothing(n, x0, P, options = (limit_stack_depth = k, ));

    buffer    = Vector{Marginal}(undef, n)
    marginals = getmarginals(x)
    
    subscription = subscribe!(marginals, (ms) -> copyto!(buffer, ms))
    
    update!(y, data)
    
    unsubscribe!(subscription)
    
    return buffer
end

inference (generic function with 1 method)

In [5]:
@benchmark res = inference(
    data = $data,
    k = $k,
    x0 = NormalMeanVariance(0.0, 10000.0),
    P = $P
)

BenchmarkTools.Trial: 
  memory estimate:  281.18 MiB
  allocs estimate:  4451255
  --------------
  minimum time:     377.763 ms (0.00% GC)
  median time:      743.603 ms (45.97% GC)
  mean time:        702.424 ms (41.98% GC)
  maximum time:     910.046 ms (53.42% GC)
  --------------
  samples:          8
  evals/sample:     1

In [6]:
@benchmark res = inference(
    data = $data,
    k = $k,
    x0 = NormalMeanVariance(0.0, 10000.0),
    P = $P
)

BenchmarkTools.Trial: 
  memory estimate:  281.18 MiB
  allocs estimate:  4451255
  --------------
  minimum time:     385.507 ms (0.00% GC)
  median time:      737.649 ms (42.80% GC)
  mean time:        668.747 ms (38.79% GC)
  maximum time:     815.623 ms (50.64% GC)
  --------------
  samples:          8
  evals/sample:     1